In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import folium
import numpy as np
import matplotlib as mpl
from folium.plugins import MarkerCluster
import json
import fiona 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.ticker as ticker

In [2]:
#import WAZE data
df=pd.read_csv('MORA_alerts.csv', encoding="ISO-8859-1")

C:\Users\Meredith.Raymer\Anaconda3\envs\geo-env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#opening link locations shapefile
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
links = gpd.read_file('FOTSCLBS Corresponding Polygons.kml', driver='KML')

In [4]:
#converting WAZE data fram into GeoDataFrame with geometry point 
alertPoints = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.location_lon, df.location_lat),crs={'init' :'epsg:4326'})
alertPoints=alertPoints.to_crs(epsg=4326)

C:\Users\Meredith.Raymer\Anaconda3\envs\geo-env\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
#clipping WAZE points to those within the entrance
MORAalertsLBS=gpd.clip(alertPoints,links)

In [6]:
alertsAlongLinks=gpd.sjoin(MORAalerts,links,how="inner", op='intersects')

In [10]:
#Number of Total Alerts at Each link
linkAlerts=alertsAlongLinks.groupby(['Name']).count()
linkAlerts=linkAlerts.reset_index()
linkAlerts=linkAlerts[['Name','alert_char_crc']].sort_values(by='Name')

In [11]:
#Adding FOTSC & LBS traffic counts to # alerts dataframe
d = {'Name': ['East/West Nisqually','SR 410 N of Sunrise Rd','SR 410 S of Sunrise Rd','Sunrise Rd. W of SR 410','West/East Stevens Canyon Rd. West of SR123'],
     'FOTSC': [275453, 337421, 291879, 163190, 137699],
    'LBS Generated':[16641, 14844, 11013, 10399, 8056]}
flows = pd.DataFrame(data=d)
alertsAndFlows=linkAlerts.join(flows.set_index('Name'), on='Name')
alertsAndFlows

,Name,alert_char_crc,FOTSC,LBS Generated
0,East/West Nisqually,730,275453,16641
1,SR 410 N of Sunrise Rd,1,337421,14844
2,SR 410 S of Sunrise Rd,2454,291879,11013
3,Sunrise Rd. W of SR 410,180,163190,10399
4,West/East Stevens Canyon Rd. West of SR123,346,137699,8056
